In [3]:
!pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


make beautifulsoup object of website

In [4]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
import bs4
import requests

r  = requests.get(url)

data = r.text

soup = bs4.BeautifulSoup(data, 'lxml')

convert website contents to dataframe

In [5]:
import pandas as pd
my_table = soup.find('table',{'class':'wikitable sortable'})
entries = my_table.find_all('td')

cols = ['postcode', 'borough', 'neighborhood']
data_dict = {'postcode': [], 'borough': [], 'neighborhood': []}

it = iter(entries)

while True:
    for col in cols:
        entry = next(it, False)
        if not entry:
            break
        if entry.string is None:
            data_dict[col].append(entry.contents[0].attrs['title'])
        else:
            data_dict[col].append(entry.string)
    else:
        continue
    
    break

df = pd.DataFrame(data_dict)
df = df[cols]

print(df[:10])

  postcode           borough            neighborhood
0      M1A      Not assigned           Not assigned

1      M2A      Not assigned           Not assigned

2      M3A        North York               Parkwoods
3      M4A        North York        Victoria Village
4      M5A  Downtown Toronto  Harbourfront (Toronto)
5      M5A  Downtown Toronto             Regent Park
6      M6A        North York        Lawrence Heights
7      M6A        North York          Lawrence Manor
8      M7A      Queen's Park           Not assigned

9      M8A      Not assigned           Not assigned



take out the rows with a borough that is not assigned

In [6]:
df = df[df.borough != 'Not assigned']
print(df[:10])

   postcode           borough            neighborhood
2       M3A        North York               Parkwoods
3       M4A        North York        Victoria Village
4       M5A  Downtown Toronto  Harbourfront (Toronto)
5       M5A  Downtown Toronto             Regent Park
6       M6A        North York        Lawrence Heights
7       M6A        North York          Lawrence Manor
8       M7A      Queen's Park           Not assigned

10      M9A         Etobicoke        Islington Avenue
11      M1B       Scarborough          Rouge, Toronto
12      M1B       Scarborough        Malvern, Toronto


merge the neighborhoods that have the same postcode

In [18]:
def tmp(group_series):
    return pd.Series(data={'borough': group_series.iloc[0].borough, 'neighborhood': ', '.join([neighborhood.strip() for neighborhood in group_series.neighborhood])})
    
df = df.groupby('postcode').agg(tmp)

# df.reset_index(inplace=True)

print(df[:20])

              borough                                       neighborhood
postcode                                                                
M1B       Scarborough                   Rouge, Toronto, Malvern, Toronto
M1C       Scarborough  Highland Creek (Toronto), Rouge Hill, Port Uni...
M1E       Scarborough  Guildwood, Morningside, Toronto, West Hill, To...
M1G       Scarborough                                    Woburn, Toronto
M1H       Scarborough                                          Cedarbrae
M1J       Scarborough                                Scarborough Village
M1K       Scarborough  East Birchmount Park, Ionview, Kennedy Park, T...
M1L       Scarborough  Clairlea, Golden Mile, Toronto, Oakridge, Toronto
M1M       Scarborough  Cliffcrest, Cliffside, Toronto, Scarborough Vi...
M1N       Scarborough                        Birch Cliff, Cliffside West
M1P       Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
M1R       Scarborough                Maryvale, Toro

if the neighborhood is not assigned but the borough is, set the neighborhood to the borough

In [19]:
df.loc[df.neighborhood == 'Not assigned', 'neighborhood'] = df.loc[df.neighborhood == 'Not assigned', 'borough']

In [20]:
df.shape

(103, 2)

non-working code to get coordinates through geocoder

In [10]:
# !pip install geocoder

In [16]:
# import geocoder # import geocoder

# latitude_list = []
# longitude_list = []

# # initialize your variable to None

# for postcode in df.postcode:
#     lat_lng_coords = None

#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         print('trying')
#         g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
#         lat_lng_coords = g.latlng
    
#     print(postcode, lat_lng_coords)

#     latitude_list.append(lat_lng_coords[0])
#     longitude_list.append(lat_lng_coords[1])

# print(latitude_list, longitude_list)

get latitudes and longitudes from csv-file

In [24]:
df_lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
df_lat_long.set_index('Postal Code', inplace=True)
print(df_lat_long.head())


              Latitude  Longitude
Postal Code                      
M1B          43.806686 -79.194353
M1C          43.784535 -79.160497
M1E          43.763573 -79.188711
M1G          43.770992 -79.216917
M1H          43.773136 -79.239476


In [28]:
df_merged = pd.concat([df, df_lat_long], axis=1)
print(df_merged.head())
print(df_merged.columns, df_merged.shape)

         borough                                       neighborhood  \
M1B  Scarborough                   Rouge, Toronto, Malvern, Toronto   
M1C  Scarborough  Highland Creek (Toronto), Rouge Hill, Port Uni...   
M1E  Scarborough  Guildwood, Morningside, Toronto, West Hill, To...   
M1G  Scarborough                                    Woburn, Toronto   
M1H  Scarborough                                          Cedarbrae   

      Latitude  Longitude  
M1B  43.806686 -79.194353  
M1C  43.784535 -79.160497  
M1E  43.763573 -79.188711  
M1G  43.770992 -79.216917  
M1H  43.773136 -79.239476  
Index(['borough', 'neighborhood', 'Latitude', 'Longitude'], dtype='object') (103, 4)
